In [ ]:
from pyspark.sql import SparkSession #Importamos lo necesario
from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType, IntegerType, StringType, FloatType
from pyspark.sql import functions as F


In [ ]:
spark = SparkSession.builder.appName('CADS').getOrCreate() #Ingresamos a la sesión de Spark
products = spark.read.json("../common/data/products.json") #Abrimos el archivo Productos
products.show()

In [ ]:
products = products.drop('imUrl', '_corrupt_record', 'brand', 'description', 'salesRank') #Tumbamos las columnas que no sirven
products = products.selectExpr('asin as productId','categories as categories','related as related','title as title','price as price') #Renombramos las columnas
products = products.dropna(subset='productId') #Quitamos los productos no iDentificados

In [ ]:
products.printSchema() #df.info()
products.show() #df.head(20)

In [ ]:
Dict_Null = {col:products.filter(products[col].isNull()).count() for col in products.columns} #Armamos un diccionario que cuenta los nulos por columna
Dict_Null #Printeamos las cuentas de nulos por columna

In [ ]:
#Cambiamos el tipo de dato de las columnas que nesecitamos.
products = products.withColumn('categories', F.col('categories').cast(StringType()))
products = products.withColumn('related', F.col('related').cast(StringType()))
products = products.withColumn('price', F.col('price').cast(FloatType()))

In [ ]:
#Normalizamos los datos
products = products.withColumn("categories",translate("categories","[",""))
products = products.withColumn("categories",translate("categories","]",""))
products = products.withColumn("related",translate("related","[",""))
products = products.withColumn("related",translate("related","]",""))


In [ ]:
products.write.json('../common/data/products_etl') #Guardamos el DataFrame